In [ ]:
# adapted from https://github.com/globus/globus-jupyter-notebooks/blob/master/Automation_Using_Globus_Flows.ipynb

In [2]:
import sys
import os
import time
import json
import uuid
import pickle
import base64

import globus_sdk
import globus_sdk.scopes

# ID of this tutorial notebook as registered with Globus Auth
CLIENT_ID = 'f794186b-f330-4595-b6c6-9c9d3e903e47'

In [ ]:
# Do a native app authentication flow to get tokens that allow us to interact with the Globus Flows service
scopes = [
    "openid",
    "profile",
    "email",
    # transfer_scope,
    globus_sdk.FlowsClient.scopes.manage_flows,
    globus_sdk.FlowsClient.scopes.run_manage,
]
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
native_auth_client.oauth2_start_flow(requested_scopes=scopes)
print(f"Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}")

# Authenticate and come back with your authorization code; paste it into the prompt below.
auth_code = input('Authorization Code: ')
response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)

# Save the new token in a place where the flows client can retrieve it.
tokens = response.by_resource_server

# These are the saved scopes for the flow
print(json.dumps(tokens, indent=2))

# Setup the Flow client, using tokens from our Jupyterhub login to access the Globus Flows service, and
# set the `get_flow_authorizer` callback for any new flows we authorize.
flows_authorizer = globus_sdk.AccessTokenAuthorizer(access_token=tokens['flows.globus.org']['access_token'])
flows_client = globus_sdk.FlowsClient(authorizer=flows_authorizer)

In [ ]:
# Create an Auth client so we can look up identities
auth_authorizer = globus_sdk.AccessTokenAuthorizer(access_token=tokens['auth.globus.org']['access_token'])
ac = globus_sdk.AuthClient(authorizer=auth_authorizer)

# Get the user's primary identity
primary_identity = ac.oauth2_userinfo()
identity_id = primary_identity['sub']

print(f"Username: {primary_identity['preferred_username']} (ID: {identity_id})")
print(f"Notifications will be sent to: {primary_identity['email']}")

In [ ]:
action_url = "https://flask-service.ml22sevubfnks.us-east-1.cs.amazonlightsail.com/"


# Define flow for sending messages to Diaspora
flow_definition = {
    "Comment": "Publish messages to Diaspora Event Fabric",
    "StartAt": "PublishMessages",
    "States": {
        "PublishMessages": {
            "Comment": "Send messages to a specified topic in Diaspora",
            "Type": "Action",
            "ActionUrl": action_url,
            "Parameters": {
                "topic.$": "$.input.topic",
                "messages.$": "$.input.messages",
                "bootstrap_servers": "b-1-public.diaspora.fy49oq.c9.kafka.us-east-1.amazonaws.com:9198,b-2-public.diaspora.fy49oq.c9.kafka.us-east-1.amazonaws.com:9198"
            },
            "ResultPath": "$.PublishMessages",
            "End": True
        }
    }
}


In [ ]:
# Define input schema for sending messages to Diaspora
input_schema = {
    "required": ["input"],
    "properties": {
        "input": {
            "type": "object",
            "required": ["topic", "messages"],
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic under which the message is published"
                },
                "messages": {
                    "type": "array",
                    "description": "List of message values, each formatted as a JSON string",
                    "items": {
                        "type": "string",
                        "contentMediaType": "application/json"
                    }
                },
                "bootstrap_servers": {
                    "type": "string",
                    "description": "A comma-separated list of bootstrap servers for Kafka connection",
                    "default": "b-1-public.diaspora.fy49oq.c9.kafka.us-east-1.amazonaws.com:9198,b-2-public.diaspora.fy49oq.c9.kafka.us-east-1.amazonaws.com:9198"
                }
            },
            "additionalProperties": False
        }
    },
    "additionalProperties": False
}

In [ ]:
# Create the flow
# Set the flow's title so you can easily identify it
flow_title = f"Diaposra-AP-{str(uuid.uuid4())[:4]}"
flow = flows_client.create_flow(
    title=flow_title,
    definition=flow_definition,
    input_schema=input_schema,
)
flow_id = flow['id']
flow_scope = globus_sdk.SpecificFlowClient(flow_id).scopes.make_mutable("user")
# flow_scope.add_dependency(transfer_scope)


"""
# If you change the flow, you will need to update it.
# For example, to make this flow visibe to another user:
flow = flows_client.update_flow(
    flow_id=flow_id, 
    flow_viewers=[f"urn:globus:auth:identity:{identity_id}"]),
)
"""

print(f"Successfully created flow: '{flow_title}'")
print(f"(ID: {flow_id})")
print(f"Flow scope: {flow_scope}\n\n")
print(f"View the flow in the Web App: https://app.globus.org/flows/{flow_id}")
print(f"Note: You can start your flow directly from the Web App")

In [ ]:
if flow_id not in tokens:
    # Do a native app authentication flow and get tokens that include the newly deployed flow scope
    native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
    native_auth_client.oauth2_start_flow(requested_scopes=flow_scope)
    print(f"Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}")
    
    # Authenticate and come back with your authorization code; paste it into the prompt below.
    auth_code = input('Authorization Code: ')
    token_response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)
    
    # Save the new token in a place where the flows client can retrieve it.
    tokens[flow_id] = token_response.by_resource_server[flow_id]
    
    # These are the saved scopes for the flow
    print(json.dumps(tokens, indent=2))

In [ ]:
from diaspora_event_sdk import Client

c = Client()
topic = "topic" + c.subject_openid[-12:]
print(c.register_topic(topic))

In [ ]:
# Define flow inputs for sending messages to Diaspora
flow_input = {
    "input": {
        "topic": topic,
        "messages": [
            '{"event": "1: system reboot", "time": "2024-05-12T08:00:00Z"}',
            '{"event": "2: maintenance start", "time": "2024-05-12T23:00:00Z"}'
        ]
    }
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Transfer/Share tutorial run for {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
  body=flow_input,
  label=run_label,
  tags=['tutorial', 'diaspora']
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print("This flow can be monitored in the Web App:")
print(f"https://app.globus.org/runs/{run_id}")
print(f"Flow run started with ID: {run_id} - Status: {run_status}")

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')
    
# Run completed
print(json.dumps(run.data, indent=2))